**Initialize database**

In [130]:
set role postgres;

CREATE EXTENSION IF NOT EXISTS pgcrypto;

/* Drop Tables */



DROP TABLE IF EXISTS accident_archive;
DROP TABLE IF EXISTS contract_archive;
DROP TABLE IF EXISTS raw_accidents;
DROP TABLE IF EXISTS raw_contracts;
DROP TABLE IF EXISTS cars;
DROP TABLE IF EXISTS clients;
DROP TABLE IF EXISTS managers;
DROP TABLE IF EXISTS parkings;
DROP TABLE IF EXISTS users;



/* Create Tables */

CREATE TABLE raw_accidents
(
	ID_accident int NOT NULL UNIQUE,
	report_date date DEFAULT now() NOT NULL,
	is_confirmed boolean DEFAULT 'FALSE' NOT NULL,
	driving_license char(10) NOT NULL,
	vin_number char(17) NOT NULL,
	PRIMARY KEY (ID_accident)
) WITHOUT OIDS;


CREATE TABLE contract_archive
(
	ID_record serial UNIQUE,
	ID_contract int NOT NULL,
	start_date date NOT NULL,
	end_date date NOT NULL,
	is_returned boolean DEFAULT 'FALSE' NOT NULL,
	total_price money NOT NULL,
	car_vin_number char(17) NOT NULL,
	client_driving_license char(10) NOT NULL,
	client_name varchar NOT NULL,
	phone_number varchar NOT NULL,
	PRIMARY KEY (ID_record)
) WITHOUT OIDS;


CREATE TABLE accident_archive
(
	ID_record serial UNIQUE,
	ID_accident int NOT NULL,
	report_date date NOT NULL,
	car_vin_number char(17) NOT NULL,
	client_driving_license char(10) NOT NULL,
	client_name varchar NOT NULL,
	phone_number varchar NOT NULL,
	PRIMARY KEY (ID_record)
) WITHOUT OIDS;


CREATE TABLE cars
(
	vin_number char(17) NOT NULL UNIQUE,
	cost_per_hour money,
	car_brand varchar NOT NULL,
	car_color varchar NOT NULL,
	car_class varchar NOT NULL,
	transmission_type varchar NOT NULL,
	fuel_type varchar NOT NULL,
	locked boolean DEFAULT 'FALSE' NOT NULL,
	ID_parking int NOT NULL,
	PRIMARY KEY (vin_number)
) WITHOUT OIDS;


CREATE TABLE clients
(
	driving_license char(10) NOT NULL,
	name varchar NOT NULL,
	phone varchar UNIQUE,
	email varchar UNIQUE,
	date_of_birth date,
	registration_address varchar,
	residence_address varchar,
	username varchar NOT NULL UNIQUE,
	PRIMARY KEY (driving_license)
) WITHOUT OIDS;


CREATE TABLE managers
(
	manager_passport_details char(10) NOT NULL UNIQUE,
	name varchar NOT NULL,
	phone varchar NOT NULL UNIQUE,
	expirience smallint DEFAULT 0,
	salary int,
	username varchar NOT NULL UNIQUE,
	ID_parking int NOT NULL,
	PRIMARY KEY (manager_passport_details)
) WITHOUT OIDS;


CREATE TABLE parkings
(
	ID_parking int NOT NULL UNIQUE,
	address varchar NOT NULL,
	PRIMARY KEY (ID_parking)
) WITHOUT OIDS;


CREATE TABLE raw_contracts
(
	ID_contract int NOT NULL UNIQUE,
	rental_duration interval NOT NULL,
	total_price money NOT NULL,
	driving_license char(10) NOT NULL UNIQUE,
	vin_number char(17) NOT NULL UNIQUE,
	signed boolean DEFAULT 'FALSE' NOT NULL,
	PRIMARY KEY (ID_contract)
) WITHOUT OIDS;


CREATE TABLE users
(
	username varchar NOT NULL UNIQUE,
	password varchar NOT NULL,
	role varchar NOT NULL,
	PRIMARY KEY (username)
) WITHOUT OIDS;



/* Create Foreign Keys */

ALTER TABLE raw_accidents
	ADD FOREIGN KEY (vin_number)
	REFERENCES cars (vin_number)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE raw_accidents
	ADD FOREIGN KEY (driving_license)
	REFERENCES clients (driving_license)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE raw_contracts
	ADD FOREIGN KEY (vin_number)
	REFERENCES cars (vin_number)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE raw_contracts
	ADD FOREIGN KEY (driving_license)
	REFERENCES clients (driving_license)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE cars
	ADD FOREIGN KEY (ID_parking)
	REFERENCES parkings (ID_parking)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE managers
	ADD FOREIGN KEY (ID_parking)
	REFERENCES parkings (ID_parking)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE clients
	ADD FOREIGN KEY (username)
	REFERENCES users (username)
	ON UPDATE CASCADE
	ON DELETE CASCADE
;


ALTER TABLE managers
	ADD FOREIGN KEY (username)
	REFERENCES users (username)
	ON UPDATE CASCADE
	ON DELETE CASCADE
;

SET

NOTICE: extension "pgcrypto" already exists, skipping

CREATE EXTENSION

DROP TABLE

DROP TABLE

...

ALTER TABLE

ALTER TABLE

ALTER TABLE

ALTER TABLE

Total execution time: 00:00:00.331

**Types**

In [131]:
-- drop type if exists type_transmission;

-- CREATE TYPE type_transmission AS ENUM ('A', 'M');

Total execution time: 00:00:00

**Alter table**

In [132]:
alter table cars
alter column transmission_type type type_transmission using transmission_type::type_transmission; 

ALTER TABLE

Total execution time: 00:00:00.022

**Functions**

In [133]:
drop function if exists check_auth;

drop function if exists archive_contract;
drop function if exists archive_accident;
drop function if exists add_new_user_role;
drop function if exists modify_on_insert;
drop function if exists get_cars;
drop function if exists add_contract;
drop function if exists add_accident;
drop function if exists create_user_role;
drop function if exists get_raw_contracts;
drop function if exists get_raw_accidents;
drop function if exists get_return;



create function check_auth(input_username varchar, input_password varchar)
returns varchar as 
$$
declare 
    user_role varchar;
begin
    select role into user_role from users where username = input_username and password = crypt(input_password, '$1$Ur3S/3Ln');
    return user_role;
end
$$ LANGUAGE plpgsql;


create function get_cars() 
returns table (hour_cost money, classs varchar, brand varchar, color varchar, transmission type_transmission, fuel varchar, vin char(17)) as $$
begin 
    return query 
        select cost_per_hour, car_class, car_brand, car_color, transmission_type, fuel_type, vin_number
        from cars 
        where locked = false;
end;
$$ language plpgsql;


create function get_raw_contracts() 
returns table (id_contract int, client_data text, car_data text, rental_duration numeric, total_price money) as $$
begin 
    return query 
        select 
            rc.ID_contract, 
            (cl.driving_license || ' ' || cl.name || ' ' || cl.phone) as client_data, 
            (c.car_brand || ' ' || c.car_color || ' ' || c.car_class) as car_data, 
            extract(hour from rc.rental_duration), 
            rc.total_price
        from raw_contracts as rc
        join cars as c on rc.vin_number = c.vin_number
        join clients as cl on rc.driving_license = cl.driving_license;
end;
$$ language plpgsql;


create function get_raw_accidents() 
returns table (id_accident int, client_data text, car_data text) as $$
begin 
    return query 
        select 
            rac.ID_accident, 
            (cl.driving_license || ' ' || cl.name || ' ' || cl.phone) as client_data, 
            (c.car_brand || ' ' || c.car_color || ' ' || c.car_class) as car_data
        from raw_accidents as rac
        join cars as c on rac.vin_number = c.vin_number
        join clients as cl on rac.driving_license = cl.driving_license;
end;
$$ language plpgsql;


create function get_return() 
returns table (id int, vin text, car_data text, client_data text) as $$
begin 
    return query 
        select 
            ca.ID_record,
            cast(ca.car_vin_number as text), 
            (c.car_brand || ' ' || c.car_color || ' ' || c.car_class) as car_data, 
            (cl.driving_license || ' ' || cl.name || ' ' || cl.phone) as client_data 
        from contract_archive as ca
        join cars as c on ca.car_vin_number = c.vin_number
        join clients as cl on cl.driving_license = ca.client_driving_license
        where ca.is_returned = false;
end;
$$ language plpgsql;


-- "Procedure-like" functions
create function add_contract(user_name varchar, vin varchar, duration interval)
returns varchar as $$
declare
    id int;
    hour_cost money;
    license varchar;
begin
    select floor(random() * 10000) into id;
    select cost_per_hour into hour_cost from cars where cars.vin_number = vin;
    select driving_license into license from clients where username = user_name;

    insert into raw_contracts values (id, duration, (hour_cost * extract(hour from duration)), license, vin);
    return id;
end;
$$ language plpgsql;


create function add_accident(user_name varchar, vin varchar)
returns varchar as $$
declare
    id int;
    license varchar;
begin
    select floor(random() * 10000) into id;
    select driving_license into license from clients where user_name = username;
    insert into raw_accidents values (id, default, default, license, vin);
    return id;
end;
$$ language plpgsql;



-- Trigger functions
create function archive_contract() returns trigger as $$
declare
    client_name varchar;
    phone_number varchar;
begin
    if new.signed = true then
        select name into client_name from clients as c where c.driving_license = new.driving_license;
        select phone into phone_number from clients as c where c.driving_license = new.driving_license;

        insert into contract_archive (ID_contract, start_date, end_date, is_returned, total_price, car_vin_number, client_driving_license, client_name, phone_number) 
        values (new.ID_contract, now(), now() + new.rental_duration, DEFAULT, new.total_price, new.vin_number, new.driving_license, client_name, phone_number);

        delete from raw_contracts where signed = true;
        return new;
    end if;
end;
$$ language plpgsql;

create function archive_accident() returns trigger as $$
declare
    client_name varchar;
    phone_number varchar;
begin
    if new.is_confirmed = true then
        select name into client_name from clients as c where c.driving_license = new.driving_license;
        select phone into phone_number from clients as c where c.driving_license = new.driving_license;

        insert into accident_archive (ID_accident, report_date, car_vin_number, client_driving_license, client_name, phone_number) 
        values (new.ID_accident, new.report_date, new.vin_number, new.driving_license, client_name, phone_number);

        delete from raw_accidents where is_confirmed = true;
        return new;
    end if;
end;
$$ language plpgsql;

create function create_user_role() returns trigger as $$
begin
    if not exists (select rolname from pg_roles where new.username = rolname) then
        execute 'create user "'||new.username||'" with inherit login connection limit 1;';
        execute 'grant _client to "'||new.username||'"';
    end if;
    return new;
end;
$$ language plpgsql;

create function modify_on_insert() returns trigger as $$
begin
    update cars set locked = true where cars.vin_number = new.vin_number;
    return new;
end;
$$ language plpgsql;

DROP FUNCTION

DROP FUNCTION

DROP FUNCTION

NOTICE: function add_new_user_role() does not exist, skipping

DROP FUNCTION

...

CREATE FUNCTION

CREATE FUNCTION

CREATE FUNCTION

CREATE FUNCTION

Total execution time: 00:00:00.089

**Procedures**

In [134]:
drop procedure if exists add_user;
drop procedure if exists add_manager;
drop procedure if exists confirm_accident;
drop procedure if exists confirm_contract;
drop procedure if exists return_car;

create procedure add_user (user_name varchar, user_password varchar) as
$$
begin
    if not exists (select username from users where user_name = username) then
        insert into users values (user_name, crypt(user_password, '$1$Ur3S/3Ln'), 'client');
    end if;
end;
$$ language plpgsql;


create procedure add_manager (user_name varchar, user_password varchar) as
$$
begin
    if not exists (select rolname from pg_roles where user_name = rolname) then
        execute 'create user "'||user_name||'" with login connection limit 1;';
        execute 'grant _manager to "'||user_name||'"';
    end if;

    if not exists (select username from users where user_name = username) then
        insert into users values (user_name, crypt(user_password, '$1$Ur3S/3Ln'), 'manager');
    end if;
end;
$$ language plpgsql;


create procedure confirm_contract (id int) as 
$$
begin 
    update raw_contracts set signed = true where ID_contract = id;
end;
$$ language plpgsql;


create procedure confirm_accident (id int) as 
$$
begin 
    update raw_accidents set is_confirmed = true where id = ID_accident;
end;
$$ language plpgsql;


create procedure return_car(id int, vin varchar) as 
$$
declare
    parking_id int;
begin 
    select ID_parking into parking_id from managers as m where m.username = current_user;
    update cars set locked = false where vin_number = vin;
    update cars set ID_parking = parking_id where vin_number = vin;

    update contract_archive set is_returned = true where ID_record = id;
end;
$$ language plpgsql;

DROP PROCEDURE

DROP PROCEDURE

DROP PROCEDURE

DROP PROCEDURE

DROP PROCEDURE

...

CREATE PROCEDURE

CREATE PROCEDURE

CREATE PROCEDURE

CREATE PROCEDURE

Total execution time: 00:00:00.040

**Triggres**

In [135]:
CREATE OR REPLACE TRIGGER contract_signed 
    AFTER UPDATE ON raw_contracts
    FOR EACH ROW
    EXECUTE PROCEDURE archive_contract();

create or replace trigger accident_confirmed
    after update on raw_accidents
    for each row
    execute procedure archive_accident();

create or replace trigger on_insert_raw_contracts
    after insert on raw_contracts
    for each row 
    execute procedure modify_on_insert();

create or replace trigger user_added
    after insert on users
    for each row
    execute procedure create_user_role();

CREATE TRIGGER

CREATE TRIGGER

CREATE TRIGGER

CREATE TRIGGER

Total execution time: 00:00:00.025

**Roles**

In [136]:
/*
*/
drop role if exists _server;
drop role if exists _manager;
drop role if exists _client;
drop role if exists selector;


-- Create roles

create role selector with login password 'selectorpassword';
grant select on all tables in schema public to selector;


create role _manager with inherit nologin;
create role _client with inherit nologin;




------------------------------------------

-- Manager configuration
grant select on cars, managers, contract_archive, accident_archive, raw_contracts, raw_accidents, clients to _manager;
grant insert on contract_archive, accident_archive to _manager;
grant update on raw_contracts, raw_accidents, cars, contract_archive to _manager;
grant delete on raw_contracts, raw_accidents to _manager;

grant execute on function get_raw_contracts, get_raw_accidents, get_return to _manager;
grant execute on procedure confirm_contract, confirm_accident to _manager;

grant usage on sequence contract_archive_id_record_seq to _manager;
grant usage on sequence accident_archive_id_record_seq to _manager;


-- Client configuration
grant select on cars, clients to _client;
grant insert on raw_contracts, raw_accidents to _client;

grant execute on function get_cars, add_contract, add_accident to _client;

NOTICE: role "_server" does not exist, skipping

DROP ROLE

DROP ROLE

DROP ROLE

DROP ROLE

...

GRANT

GRANT

GRANT

GRANT

Total execution time: 00:00:00.078

**Polices**

In [137]:
set role postgres;
-- alter table cars enable row level security;
alter table raw_accidents enable row level security;
alter table raw_contracts enable row level security;

-- client 


-- manager
create policy manager_update_raw_contracts on raw_contracts
    for update to _manager 
    using (true);

create policy manager_delete_raw_contracts on raw_contracts
    for delete to _manager 
    using (true);

create policy manager_update_raw_accidents on raw_accidents
    for update to _manager 
    using (true);

create policy manager_delete_raw_accidents on raw_accidents
    for delete to _manager 
    using (true);

create policy manager_select_raw_accidents_policy on raw_accidents
    for select to _manager
    using (vin_number in (
        select c.vin_number from cars as c where c.id_parking = (
            select ID_parking from managers where username = current_user
        )
    ));

create policy manager_select_raw_contracts_policy on raw_contracts
    for select to _manager
    using (vin_number in (
        select c.vin_number from cars as c where c.id_parking = (
            select ID_parking from managers where username = current_user
        )
    ));


SET

ALTER TABLE

ALTER TABLE

CREATE POLICY

CREATE POLICY

CREATE POLICY

CREATE POLICY

CREATE POLICY

CREATE POLICY

Total execution time: 00:00:00.031

Index

In [138]:
CREATE UNIQUE INDEX index_cars ON cars (vin_number);

CREATE INDEX

Total execution time: 00:00:00.012

Fill

In [139]:
delete from parkings;
delete from managers;
delete from clients;
delete from raw_accidents;
delete from cars;
delete from users;


drop role if exists karpovato2611;
drop role if exists ivanovep1234;
drop role if exists petrovdm2345;
drop role if exists sidorovama9999;
drop role if exists kuznetsovks3355;
drop role if exists novikovaod8899;
drop role if exists smirnovsp4123;
drop role if exists polyakovaai8998;
drop role if exists vasilievva7777;
drop role if exists kozlovanv6778;
drop role if exists nikitinaes8877;
drop role if exists alekseevaa8855;
drop role if exists maximovam1111;
drop role if exists fedorovaf6666;
drop role if exists egorova7688;
drop role if exists dmitrievai3629;

call add_user('karpovato2611', 'password2611');
call add_user('ivanovep1234', 'password1234');
call add_user('petrovdm2345', 'password2345');
call add_user('sidorovama9999', 'password9999');
call add_user('kuznetsovks3355', 'password3355');
call add_user('novikovaod8899', 'password8899');
call add_user('smirnovsp4123', 'password4123');
call add_user('polyakovaai8998', 'password8998');
call add_user('vasilievva7777', 'password7777');
call add_user('kozlovanv6778', 'password6778');
call add_user('nikitinaes8877', 'password8877');
call add_user('alekseevaa8855', 'password8855');
call add_user('maximovam1111', 'password1111');
call add_user('fedorovaf6666', 'password6666');
call add_user('egorova7688', 'password7688');
call add_user('dmitrievai3629', 'password3629');

INSERT INTO parkings (ID_parking, address) VALUES 
(1, 'г. Москва, Улица Тверская, дом 10'),
(2, 'г. Москва, Проезд Нижегородская, дом 7'),
(3, 'г. Москва, Ленинский проспект, дом 45'),
(4, 'г. Москва, Большая Спасская улица, дом 12');

INSERT INTO clients (driving_license, name, phone, email, date_of_birth, registration_address, residence_address, username) VALUES 
('7414292010', 'Карпова Татьяна Олеговна', '89955883514', 'karpova.tatyana1976', '1976-03-24', 'г. Москва, ул. Карла-Маркса, д. 13', 'г. Москва, ул. Карла-Маркса, д. 13', 'karpovato2611'),
('5648612345', 'Иванов Егор Павлович', '89951234567', 'ivanov.egor@mail.ru', '1987-05-12', 'г. Москва, ул. Ленина, д. 5', 'г. Москва, ул. Ленина, д. 5', 'ivanovep1234'),
('4589623471', 'Петров Денис Максимович', '89957682345', 'petrov.denis@mail.ru', '1995-08-21', 'г. Санкт-Петербург, ул. Приморская, д. 10', 'г. Москва, ул. Пушкина, д. 10', 'petrovdm2345'),
('6487239641', 'Сидорова Мария Александровна', '89959999999', 'sidorova.maria@mail.ru', '1989-02-17', 'г. Москва, ул. Пушкина, д. 25', 'г. Москва, ул. Пушкина, д. 25', 'sidorovama9999'),
('3578927416', 'Кузнецов Константин Сергеевич', '89953333555', 'kuznetsov.konstantin@mail.ru', '1976-12-04', 'г. Новосибирск, ул. Мира, д. 66', 'г. Москва, ул. Московская, д. 6', 'kuznetsovks3355'),
('1426958379', 'Новикова Ольга Дмитриевна', '89957778899', 'novikova.olga@mail.ru', '1975-10-31', 'г. Москва, ул. Лесная, д. 3', 'г. Москва, ул. Ленина, д. 3', 'novikovaod8899'),
('3294871692', 'Смирнов Сергей Павлович', '89954441234', 'smirnov.sergey@mail.ru', '1980-07-19', 'г. Владивосток, ул. Центральная, д. 54', 'г. Москва, ул. Красная, д. 50', 'smirnovsp4123'),
('8974526813', 'Полякова Анна Игоревна', '89958889988', 'polyakova.anna@mail.ru', '1987-03-28', 'г. Казань, ул. Гагарина, д. 12', 'г. Москва, ул. Ленина, д. 2', 'polyakovaai8998'),
('2634890017', 'Васильев Владимир Александрович', '89957777777', 'vasiliev.vladimir@mail.ru', '1973-11-15', 'г. Ростов-на-Дону, ул. Советская, д. 78', 'г. Москва, ул. Светлая, д. 8', 'vasilievva7777'),
('3785910035', 'Козлова Наталья Владимировна', '89956667788', 'kozlova.natalia@mail.ru', '1986-02-25', 'г. Челябинск, ул. Ленина, д. 30', 'г. Москва, ул. Большая, д. 32', 'kozlovanv6778'),
('4567893210', 'Никитина Елена Сергеевна', '89959998877', 'nikitina.elena@mail.ru', '1990-05-08', 'г. Уфа, ул. Победы, д. 15', 'г. Москва, ул. Первая, д. 75', 'nikitinaes8877'),
('1089756423', 'Алексеев Алексей Алексеевич', '89957778855', 'alekseev.aleksei@mail.ru', '1974-08-02', 'г. Самара, ул. Ленина, д. 48', 'г. Москва, ул. Ленина, д. 6', 'alekseevaa8855'),
('8945739182', 'Максимов Артем Матвеевич', '89951111111', 'maximov.artem@mail.ru', '1972-12-24', 'г. Волgоград, ул. Пушкина, д. 7', 'г. Москва, ул. Первомайская, д. 17', 'maximovam1111'),
('7463879245', 'Федоров Александр Федорович', '89953336666', 'fedorov.alexandr@mail.ru', '1987-06-13', 'г. Омcк, ул. Мира, д. 122', 'г. Москва, ул. Маяковская, д. 22', 'fedorovaf6666'),
('5478921436', 'Егорова Екатерина Александровна', '89957778819', 'egorova.ekaterina@mail.ru', '1975-02-15', 'г. Ульяновск, ул. Советская, д. 189', 'г. Москва, ул. Марксисткая, д. 89', 'egorova7688'),
('3311248975', 'Дмитриев Андрей Игоревич', '89956453629', 'dmitriev.andrei@mail.ru', '1990-09-14', 'г. Тольятти, ул. Ленина, д. 89', 'г. Москва, ул. Калинина, д. 9', 'dmitrievai3629');

INSERT INTO cars (vin_number, cost_per_hour, car_brand, car_color, car_class, transmission_type, fuel_type, locked, ID_parking) VALUES 
('2C4GJ53XYR6936972', 900, 'Honda', 'черный', 'B', 'A', 'бензин', DEFAULT, 1),
('1HGCR2F5XFA02A996', 800, 'Toyota', 'синий', 'C', 'M', 'бензин', DEFAULT, 2),
('1HGCR2F5XFA02B001', 750, 'Hyundai', 'зеленый', 'B', 'A', 'гибрид', DEFAULT, 3),
('1HGCR2F5XFA02C002', 700, 'BMW', 'черный', 'A', 'M', 'электричество', DEFAULT, 4),
('5YJSA1CNXDFP00750', 1200, 'Tesla', 'серебристый', 'E', 'A', 'электричество', DEFAULT, 2),
('1G6AB5S3XDPR46200', 750, 'Cadillac', 'красный', 'D', 'A', 'бензин', DEFAULT, 3),
('1G1FB1RS4N0101077', 600, 'Chevrolet', 'белый', 'B', 'M', 'бензин', DEFAULT, 4),
('1G4PP5LK0E4105633', 800, 'Buick', 'синий', 'C', 'A', 'гибрид', DEFAULT, 1),
('9FAHP2HW0DG164988', 650, 'Ford', 'серый', 'E', 'M', 'бензин', DEFAULT, 2),
('2S1BM1W74E1128568', 700, 'Mazda', 'синий', 'D', 'A', 'бензин', DEFAULT, 3),
('2LMHJ5AT8HGB77688', 850, 'Lincoln', 'черный', 'E', 'A', 'бензин', DEFAULT, 4),
('1TDKN3DU4F3253596', 900, 'Toyota', 'красный', 'B', 'M', 'гибрид', DEFAULT, 1),
('5YJSA1CN2DFP00750', 1200, 'Tesla', 'зеленый', 'E', 'A', 'электричество', DEFAULT, 2),
('1G6AB1S3XDPR46369', 750, 'Cadillac', 'серый', 'D', 'M', 'бензин', DEFAULT, 3),
('2C4GJ53KR69369788', 900, 'Honda', 'белый', 'B', 'A', 'бензин', DEFAULT, 4),
('5YJSA1CN7DFP00750', 1200, 'Tesla', 'красный', 'E', 'A', 'электричество', DEFAULT, 1),
('1G6AB5S3XDPR46344', 750, 'Cadillac', 'серебристый', 'D', 'A', 'бензин', DEFAULT, 2),
('9J1FB1RS4D0801067', 600, 'Chevrolet', 'черный', 'B', 'M', 'бензин', DEFAULT, 3),
('3G4IO5UD0E4158333', 800, 'Buick', 'серый', 'C', 'A', 'гибрид', DEFAULT, 4),
('7FAHP2HW0HK168461', 650, 'Ford', 'красный', 'E', 'A', 'бензин', DEFAULT, 1),
('8YK5M1W74E1168354', 700, 'Mazda', 'синий', 'D', 'A', 'бензин', DEFAULT, 2),
('2KH7J5AT8HGB78354', 850, 'Lincoln', 'зеленый', 'E', 'M', 'бензин', DEFAULT, 3),
('5FGKN3DU4F3257321', 900, 'Toyota', 'белый', 'B', 'A', 'гибрид', DEFAULT, 4),
('5KKSA1CN2DFP07632', 1200, 'Tesla', 'серый', 'E', 'A', 'электричество', DEFAULT, 1),
('1G6JX1S3XDPR89366', 750, 'Cadillac', 'черный', 'D', 'A', 'бензин', DEFAULT, 2),
('2C4GJ53KR69369789', 900, 'Honda', 'красный', 'B', 'M', 'бензин', DEFAULT, 3),
('5YHHR1CN7DFP76388', 1200, 'Tesla', 'серебристый', 'E', 'A', 'электричество', DEFAULT, 4),
('3G6AB5S3XDPR44569', 750, 'Cadillac', 'белый', 'D', 'A', 'бензин', DEFAULT, 1),
('8G1FB1YS4B0101967', 600, 'Chevrolet', 'синий', 'B', 'M', 'бензин', DEFAULT, 2),
('9G4WE5SK0K4100451', 800, 'Buick', 'красный', 'C', 'A', 'гибрид', DEFAULT, 3),
('8FAHP2KB0DG169056', 650, 'Ford', 'серый', 'E', 'A', 'бензин', DEFAULT, 4),
('9H1BM1W74E1156623', 700, 'Mazda', 'зеленый', 'D', 'A', 'гибрид', DEFAULT, 1),
('2GN7J5AT8HGB89440', 850, 'Lincoln', 'белый', 'E', 'M', 'бензин', DEFAULT, 2),
('6HC8N3DU4F3298541', 900, 'Toyota', 'синий', 'B', 'A', 'гибрид', DEFAULT, 3),
('5SDFA1CN2DFP66980', 1200, 'Tesla', 'черный', 'E', 'A', 'электричество', DEFAULT, 4),
('1L6AB1S3XDPR48560', 750, 'Cadillac', 'красный', 'D', 'M', 'бензин', DEFAULT, 1),
('2C4GJ53KR69369748', 900, 'Honda', 'серый', 'B', 'A', 'бензин', DEFAULT, 2),
('5YJSA1CN7LWP77351', 1200, 'Tesla', 'зеленый', 'E', 'A', 'электричество', DEFAULT, 3),
('2T6AB5S3XDPR72811', 750, 'Cadillac', 'черный', 'D', 'A', 'бензин', DEFAULT, 4),
('7G1FB1RS4C0101583', 600, 'Chevrolet', 'красный', 'B', 'M', 'бензин', DEFAULT, 1),
('2G4PP5LP0E4684333', 800, 'Buick', 'серый', 'C', 'A', 'гибрид', DEFAULT, 2),
('7FAHP2QR0DG190457', 650, 'Ford', 'зеленый', 'E', 'A', 'бензин', DEFAULT, 3),
('8I1BM1W74E1672209', 700, 'Mazda', 'белый', 'D', 'A', 'бензин', DEFAULT, 4),
('2HH6J5AT8HGB79981', 850, 'Lincoln', 'синий', 'E', 'M', 'бензин', DEFAULT, 1),
('7GUKN3DU4F3286409', 900, 'Toyota', 'черный', 'B', 'A', 'гибрид', DEFAULT, 2),
('5GHWA1CN2DFP89567', 1200, 'Tesla', 'серый', 'E', 'A', 'электричество', DEFAULT, 3),
('1Z6AB1S3XDPR33719', 750, 'Cadillac', 'красный', 'D', 'M', 'бензин', DEFAULT, 4),
('2C4GJ53KR69369796', 900, 'Honda', 'синий', 'B', 'A', 'бензин', DEFAULT, 1),
('5KJSA1DG7DFP08843', 1200, 'Tesla', 'черный', 'E', 'A', 'электричество', DEFAULT, 2),
('5KLSA1CNXDFP09365', 750, 'Cadillac', 'зеленый', 'D', 'A', 'бензин', DEFAULT, 3),
('1J6AB5S3XDPR44369', 700, 'Mazda', 'синий', 'D', 'A', 'БЕНЗИН', DEFAULT, 2),
('6NKCV54EZ4M604499', 1100, 'Mercedes-Benz', 'серый', 'C', 'A', 'гибрид', DEFAULT, 2),
('1HJHP3EN1AW538231', 800, 'Ford', 'красный', 'A', 'M', 'бензин', DEFAULT, 3),
('9M1BK12F761456789', 750, 'Mazda', 'синий', 'B', 'M', 'бензин', DEFAULT, 4),
('7H2SW22N2N1V63824', 950, 'Toyota', 'белый', 'C', 'A', 'бензин', DEFAULT, 1),
('1BAAM5338YF123456', 1200, 'BMW', 'черный', 'D', 'A', 'бензин', DEFAULT, 2),
('7AUZZZ4DX9N012345', 850, 'Audi', 'серый', 'E', 'A', 'бензин', DEFAULT, 3),
('1C1PK5SB6E7274536', 1000, 'Chevrolet', 'красный', 'A', 'A', 'бензин', DEFAULT, 4),
('3VWZZZ6RZBY123456', 950, 'Volkswagen', 'зеленый', 'B', 'M', 'бензин', DEFAULT, 1),
('4H4NA216X1T123456', 900, 'Acura', 'голубой', 'C', 'M', 'бензин', DEFAULT, 2),
('1G3NL12E73C123456', 1100, 'Oldsmobile', 'черный', 'D', 'A', 'бензин', DEFAULT, 3),
('6AULC68E34A012345', 800, 'Audi', 'серый', 'E', 'A', 'бензин', DEFAULT, 4),
('2G1WX15K529123456', 750, 'Chevrolet', 'красный', 'A', 'M', 'бензин', DEFAULT, 1),
('9VWZZZ6RZBY098765', 950, 'Volkswagen', 'зеленый', 'B', 'A', 'бензин', DEFAULT, 2),
('7P1AD29P07L097012', 1200, 'Porsche', 'белый', 'C', 'A', 'дизель', DEFAULT, 3),
('9H4DA3350LS123456', 1000, 'Acura', 'голубой', 'D', 'A', 'бензин', DEFAULT, 4),
('4A3AC79L9LU123456', 850, 'Mitsubishi', 'красный', 'A', 'A', 'бензин', DEFAULT, 1),
('2A1YD64B32N123456', 950, 'Audi', 'черный', 'B', 'M', 'бензин', DEFAULT, 2),
('2LATFY126K3123456', 900, 'Ford', 'серый', 'C', 'M', 'бензин', DEFAULT, 3),
('8H4DA9460LS123456', 1100, 'Acura', 'голубой', 'D', 'A', 'бензин', DEFAULT, 4),
('3GYFK66N6XG256789', 800, 'Cadillac', 'белый', 'E', 'A', 'гибрид', DEFAULT, 1),
('9AULC68E54A123456', 1200, 'Audi', 'черный', 'A', 'A', 'бензин', DEFAULT, 2),
('1C3CCBBBXDN666789', 750, 'Chrysler', 'серый', 'B', 'M', 'бензин', DEFAULT, 3),
('3C3CFFFH2ET123456', 950, 'Fiat', 'красный', 'C', 'M', 'бензин', DEFAULT, 4),
('8VWZZZ6RZBY543210', 900, 'Volkswagen', 'зеленый', 'D', 'A', 'бензин', DEFAULT, 1),
('8HMFA15569S789012', 1100, 'Honda', 'серый', 'E', 'A', 'гибрид', DEFAULT, 2),
('2GTEC13V271345678', 800, 'GMC', 'белый', 'A', 'A', 'бензин', DEFAULT, 3),
('3BAWB33598P012345', 750, 'BMW', 'черный', 'B', 'M', 'бензин', DEFAULT, 4),
('3T2BK18U900543210', 950, 'Toyota', 'серебристый', 'C', 'M', 'бензин', DEFAULT, 1),
('3D7TT2CT3BG345678', 1000, 'Dodge', 'красный', 'D', 'A', 'бензин', DEFAULT, 2),
('5TKJF5C77F3044994', 850, 'Scion', 'оранжевый', 'E', 'A', 'гибрид', DEFAULT, 3),
('1GTEC14V291234567', 950, 'GMC', 'зеленый', 'A', 'A', 'бензин', DEFAULT, 4),
('1FAFP58201G456789', 900, 'Ford', 'серый', 'B', 'M', 'бензин', DEFAULT, 1),
('1HGEM1157XL123456', 1100, 'Honda', 'серебристый', 'C', 'M', 'бензин', DEFAULT, 2),
('5BAAM5338XFR12345', 800, 'BMW', 'черный', 'D', 'A', 'бензин', DEFAULT, 3),
('4AJWA6FU6F8K91234', 750, 'Jaguar', 'синий', 'E', 'A', 'дизель', DEFAULT, 4),
('4TDKDTB30C1449876', 950, 'Toyota', 'оранжевый', 'A', 'A', 'бензин', DEFAULT, 1),
('1G8AL54F946123456', 900, 'Saturn', 'серый', 'B', 'M', 'бензин', DEFAULT, 2),
('1FAFP55U54G098765', 1200, 'Ford', 'красный', 'C', 'M', 'бензин', DEFAULT, 3),
('1FMCU0GX8DUC89012', 1000, 'Ford', 'зеленый', 'D', 'A', 'бензин', DEFAULT, 4),
('4MHJF35FDGY702123', 850, 'Hyundai', 'серебристый', 'E', 'A', 'бензин', DEFAULT, 1),
('1FMCU9C71CK345678', 950, 'Ford', 'черный', 'A', 'M', 'бензин', DEFAULT, 2),
('3N8AZ2KR8CT123456', 900, 'Nissan', 'синий', 'B', 'M', 'бензин', DEFAULT, 3),
('1G1ZB5E14CF123456', 1100, 'Chevrolet', 'серый', 'C', 'A', 'бензин', DEFAULT, 4),
('3VWZZZ6RZBY987654', 800, 'Volkswagen', 'черный', 'D', 'A', 'бензин', DEFAULT, 1),
('8V4940DLX81678901', 750, 'Volvo', 'белый', 'E', 'A', 'бензин', DEFAULT, 2),
('9TDKTUD32DDS61008', 950, 'Toyota', 'серый', 'A', 'A', 'гибрид', DEFAULT, 3),
('2NKCV54EZ4M604499', 1100, 'Mercedes-Benz', 'серый', 'C', 'A', 'гибрид', DEFAULT, 2),
('1FAHP3EN1AW123456', 800, 'Ford', 'красный', 'A', 'M', 'бензин', DEFAULT, 3),
('7T2SW22N2N1V65432', 950, 'Toyota', 'белый', 'C', 'A', 'бензин', DEFAULT, 1),
('3BAAM5338YF123456', 1200, 'BMW', 'черный', 'D', 'A', 'бензин', DEFAULT, 2),
('2AUZZZ4DX9N012345', 850, 'Audi', 'серый', 'E', 'A', 'гибрид', DEFAULT, 3),
('1M1PK5SB6E7776804', 1000, 'Chevrolet', 'красный', 'A', 'A', 'бензин', DEFAULT, 4),
('7VWZZZ6RZBY123456', 950, 'Volkswagen', 'зеленый', 'B', 'M', 'бензин', DEFAULT, 1),
('8H4NA216X1T123456', 900, 'Acura', 'голубой', 'C', 'M', 'бензин', DEFAULT, 2),
('1G3NL12E73C734299', 1100, 'Oldsmobile', 'черный', 'D', 'A', 'бензин', DEFAULT, 3);

DELETE 0

DELETE 0

DELETE 0

DELETE 0

DELETE 0

...

CALL

INSERT 0 4

INSERT 0 16

INSERT 0 106

Total execution time: 00:00:00.236

In [140]:
delete from raw_accidents; 

select add_accident('karpovato2611', '2C4GJ53XYR6936972');
select add_accident('ivanovep1234', '1HGCR2F5XFA02A996');
select add_accident('petrovdm2345', '1HGCR2F5XFA02B001');
select add_accident('sidorovama9999', '1HGCR2F5XFA02C002');
select add_accident('kuznetsovks3355', '5YJSA1CNXDFP00750');
select add_accident('novikovaod8899', '1G6AB5S3XDPR46200');
select add_accident('smirnovsp4123', '1G1FB1RS4N0101077');
select add_accident('polyakovaai8998', '1G4PP5LK0E4105633');
select add_accident('vasilievva7777', '9FAHP2HW0DG164988');
select add_accident('kozlovanv6778', '2S1BM1W74E1128568');

DELETE 0

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

...

add_accident
5511


add_accident
5410


add_accident
5157


add_accident
6194


add_accident
1939


In [141]:
drop role if exists petrovaai21;
drop role if exists kozlovds75;
drop role if exists smirnovae12;
drop role if exists kuznetsovma88;
drop role if exists vasilyeava44;
drop role if exists novikovia99;
drop role if exists pavlovamd01;
drop role if exists morozovpi67;
drop role if exists alexeevais23;
drop role if exists sidorovav77;

call add_manager('petrovaai21', 'password21');
call add_manager('kozlovds75', 'password75');
call add_manager('smirnovae12', 'password12');
call add_manager('kuznetsovma88', 'password88');
call add_manager('vasilyeava44', 'password44');
call add_manager('novikovia99', 'password99');
call add_manager('pavlovamd01', 'password01');
call add_manager('morozovpi67', 'password67');
call add_manager('alexeevais23', 'password23');
call add_manager('sidorovav77', 'password77');

INSERT INTO managers (manager_passport_details, name, phone, expirience, salary, username, ID_parking) VALUES 
('2451298347', 'Петрова Анна Ивановна', '79261123456', 5, 110000, 'petrovaai21', 2),
('3785621940', 'Козлов Дмитрий Сергеевич', '79257789456', 10, 140000, 'kozlovds75', 1),
('4856912740', 'Смирнова Екатерина Павловна', '79251234567', 15, 150000, 'smirnovae12', 4),
('5738291045', 'Кузнецов Максим Александрович', '79254123897', 20, 200000, 'kuznetsovma88', 3),
('6194038572', 'Васильева Александра Владимировна', '79258736745', 25, 250000, 'vasilyeava44', 2),
('7348519260', 'Новиков Игорь Артемович', '79256123984', 3, 90000, 'novikovia99', 1),
('8420365197', 'Павлова Марина Дмитриевна', '79257789012', 2, 70000, 'pavlovamd01', 4),
('9572304618', 'Морозов Павел Игоревич', '79251235678', 4, 100000, 'morozovpi67', 3),
('1057264938', 'Алексеева Ирина Сергеевна', '79251123897', 1, 55000, 'alexeevais23', 2),
('1184365927', 'Сидоров Алексей Владимирович', '79258716745', 8, 120000, 'sidorovav77', 1);

DROP ROLE

DROP ROLE

DROP ROLE

DROP ROLE

DROP ROLE

...

CALL

CALL

CALL

INSERT 0 10

Total execution time: 00:00:00.096

In [142]:
select add_contract('karpovato2611', '2C4GJ53XYR6936972', '6 hours');
select add_contract('ivanovep1234', '1HGCR2F5XFA02A996', '5 hours');
select add_contract('petrovdm2345', '1HGCR2F5XFA02B001', '8 hours');
select add_contract('sidorovama9999', '1HGCR2F5XFA02C002', '2 hours');
select add_contract('kuznetsovks3355', '5YJSA1CNXDFP00750', '5 hours');
select add_contract('novikovaod8899', '1G6AB5S3XDPR46200', '8 hours');
select add_contract('smirnovsp4123', '1G1FB1RS4N0101077', '11 hours');
select add_contract('polyakovaai8998', '1G4PP5LK0E4105633', '7 hours');
select add_contract('vasilievva7777', '9FAHP2HW0DG164988', '15 hours');
select add_contract('kozlovanv6778', '2S1BM1W74E1128568', '4 hours');
select add_contract('nikitinaes8877', '2LMHJ5AT8HGB77688', '18 hours');

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

...

add_contract
7463


add_contract
4892


add_contract
1165


add_contract
9376


add_contract
3944


In [143]:
select * from accident_archive;
select * from contract_archive;
select * from raw_accidents;
select * from raw_contracts;
select * from cars;
select * from clients;
select * from managers;
select * from parkings;
select * from users;

Server closed the connection unexpectedly. Attempting to reconnect...

(3 row(s) affected)

(7 row(s) affected)

(8 row(s) affected)

(106 row(s) affected)

...

vin_number,cost_per_hour,car_brand,car_color,car_class,transmission_type,fuel_type,locked,id_parking
1TDKN3DU4F3253596,$900.00,Toyota,красный,B,M,гибрид,False,1
5YJSA1CN2DFP00750,"$1,200.00",Tesla,зеленый,E,A,электричество,False,2
1G6AB1S3XDPR46369,$750.00,Cadillac,серый,D,M,бензин,False,3
2C4GJ53KR69369788,$900.00,Honda,белый,B,A,бензин,False,4
5YJSA1CN7DFP00750,"$1,200.00",Tesla,красный,E,A,электричество,False,1
1G6AB5S3XDPR46344,$750.00,Cadillac,серебристый,D,A,бензин,False,2
9J1FB1RS4D0801067,$600.00,Chevrolet,черный,B,M,бензин,False,3
3G4IO5UD0E4158333,$800.00,Buick,серый,C,A,гибрид,False,4
7FAHP2HW0HK168461,$650.00,Ford,красный,E,A,бензин,False,1
8YK5M1W74E1168354,$700.00,Mazda,синий,D,A,бензин,False,2


driving_license,name,phone,email,date_of_birth,registration_address,residence_address,username
7414292010,Карпова Татьяна Олеговна,89955883514,karpova.tatyana1976,1976-03-24,"г. Москва, ул. Карла-Маркса, д. 13","г. Москва, ул. Карла-Маркса, д. 13",karpovato2611
5648612345,Иванов Егор Павлович,89951234567,ivanov.egor@mail.ru,1987-05-12,"г. Москва, ул. Ленина, д. 5","г. Москва, ул. Ленина, д. 5",ivanovep1234
4589623471,Петров Денис Максимович,89957682345,petrov.denis@mail.ru,1995-08-21,"г. Санкт-Петербург, ул. Приморская, д. 10","г. Москва, ул. Пушкина, д. 10",petrovdm2345
6487239641,Сидорова Мария Александровна,89959999999,sidorova.maria@mail.ru,1989-02-17,"г. Москва, ул. Пушкина, д. 25","г. Москва, ул. Пушкина, д. 25",sidorovama9999
3578927416,Кузнецов Константин Сергеевич,89953333555,kuznetsov.konstantin@mail.ru,1976-12-04,"г. Новосибирск, ул. Мира, д. 66","г. Москва, ул. Московская, д. 6",kuznetsovks3355
1426958379,Новикова Ольга Дмитриевна,89957778899,novikova.olga@mail.ru,1975-10-31,"г. Москва, ул. Лесная, д. 3","г. Москва, ул. Ленина, д. 3",novikovaod8899
3294871692,Смирнов Сергей Павлович,89954441234,smirnov.sergey@mail.ru,1980-07-19,"г. Владивосток, ул. Центральная, д. 54","г. Москва, ул. Красная, д. 50",smirnovsp4123
8974526813,Полякова Анна Игоревна,89958889988,polyakova.anna@mail.ru,1987-03-28,"г. Казань, ул. Гагарина, д. 12","г. Москва, ул. Ленина, д. 2",polyakovaai8998
2634890017,Васильев Владимир Александрович,89957777777,vasiliev.vladimir@mail.ru,1973-11-15,"г. Ростов-на-Дону, ул. Советская, д. 78","г. Москва, ул. Светлая, д. 8",vasilievva7777
3785910035,Козлова Наталья Владимировна,89956667788,kozlova.natalia@mail.ru,1986-02-25,"г. Челябинск, ул. Ленина, д. 30","г. Москва, ул. Большая, д. 32",kozlovanv6778


manager_passport_details,name,phone,expirience,salary,username,id_parking
2451298347,Петрова Анна Ивановна,79261123456,5,110000,petrovaai21,2
3785621940,Козлов Дмитрий Сергеевич,79257789456,10,140000,kozlovds75,1
4856912740,Смирнова Екатерина Павловна,79251234567,15,150000,smirnovae12,4
5738291045,Кузнецов Максим Александрович,79254123897,20,200000,kuznetsovma88,3
6194038572,Васильева Александра Владимировна,79258736745,25,250000,vasilyeava44,2
7348519260,Новиков Игорь Артемович,79256123984,3,90000,novikovia99,1
8420365197,Павлова Марина Дмитриевна,79257789012,2,70000,pavlovamd01,4
9572304618,Морозов Павел Игоревич,79251235678,4,100000,morozovpi67,3
1057264938,Алексеева Ирина Сергеевна,79251123897,1,55000,alexeevais23,2
1184365927,Сидоров Алексей Владимирович,79258716745,8,120000,sidorovav77,1


id_parking,address
1,"г. Москва, Улица Тверская, дом 10"
2,"г. Москва, Проезд Нижегородская, дом 7"
3,"г. Москва, Ленинский проспект, дом 45"
4,"г. Москва, Большая Спасская улица, дом 12"


username,password,role
karpovato2611,$1$Ur3S/3Ln$Dc821wbBpXusU.L.OJV2G1,client
ivanovep1234,$1$Ur3S/3Ln$Hlc3/s/lFpmtAbjmvaF9N1,client
petrovdm2345,$1$Ur3S/3Ln$yAOgNZykjVuqtQulBHmzf/,client
sidorovama9999,$1$Ur3S/3Ln$x5wllImGI7dKFsh./tZV61,client
kuznetsovks3355,$1$Ur3S/3Ln$uQ0wDbIkIEpPz130HTJNw0,client
novikovaod8899,$1$Ur3S/3Ln$jEWLaq6b2QD8Ud9gY6X/j1,client
smirnovsp4123,$1$Ur3S/3Ln$9e9irWsFTegZa6Ni0MX49.,client
polyakovaai8998,$1$Ur3S/3Ln$YvGL2xNTEhK7Sx53vt9Dw1,client
vasilievva7777,$1$Ur3S/3Ln$SfV6cEE1xGopifOZBlRFO/,client
kozlovanv6778,$1$Ur3S/3Ln$CchJsbq2bRlB9wl3Lox47/,client
